<a href="https://colab.research.google.com/github/joseantbelloac/Talentpitch/blob/main/Talentpitch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Talentpitch data engineer trials

Tools preparation

In [92]:
#install pyspark
!pip install pyspark


In [93]:
#create a folder to allocate the csv files
!mkdir talentpitch


mkdir: cannot create directory ‘talentpitch’: File exists


In [94]:
#make a copy of the file to clean
!cp /content/talentpitch/"challenges - challenges.csv" ./talentpitch/challenges.csv

#Data Cleansing Process

In [95]:
#import regular expressions library to solve text anomalies in the challenges file
import re

#build a function to clean the whole data wrongly classified rows in the csv format
def challenges_cleansing(file_path):
  with open(file_path, 'r') as f:
    lines = f.readlines()
    new_lines = []
    flag=-1
    for i in range(len(lines)):
      if flag==i:
        continue
      elif lines[i][1].isnumeric() and lines[i][0]=='"' and lines[i+1][0]=='"' and not lines[i+1][1].isnumeric():
        new_line = lines[i][1:-3]+lines[i+1][1:-2]+'\n'
        new_line=re.sub('""','"',new_line)
        flag=i+1
        new_lines.append(new_line)
        continue
      elif lines[i]!=lines[0] and lines[i][0]=='"' and lines[i][1].isnumeric():
        new_line = lines[i][1:]
        new_line = new_line[:-2]
        new_line=re.sub('""','"',new_line)
        new_line=new_line+'\n'
      else:
        new_line = lines[i]
        new_line=re.sub('""','"',new_line)
      new_lines.append(new_line)
    lines=new_lines
  return lines

#build a function to delete the file
def clear_file_contents(file_path):
  with open(file_path, 'w') as f:
    f.truncate(0)


In [96]:
#use functions to clean the rows in the challenges file
file_path = "talentpitch/challenges.csv"
new_lines = challenges_cleansing(file_path)

#cleat the original file to replace with the cleaned one
clear_file_contents("talentpitch/challenges - challenges.csv")
with open("talentpitch/challenges - challenges.csv", "w") as f:
  for line in new_lines:
    f.write(line)

Data Validation

In [97]:
#import pandas library to validate the csv files format
import pandas as pd
#read each file
df_chall=pd.read_csv("talentpitch/challenges - challenges.csv")
df_prof=pd.read_csv("talentpitch/profiles - Hoja 1.csv")
df_resum=pd.read_csv("talentpitch/resumes - resumes.csv")
df_user=pd.read_csv("talentpitch/users - users.csv")

In [98]:
#preview each file to see their fields
df_user

,id,name,identification_number,slug,video,email,gender,created_at,updated_at
0,300,Pedro diaz,7769157043,pedr_diaz,https://media.talentpitch.co/resumes/1/akjdaks...,james.smith47@example.com,M,2024-02-22,2024-05-22
1,400,Juan suarez,9393968180,juanjo-suarez-2,https://media.talentpitch.co/resumes/2/dadkajd...,emma.johnson87@example.org,M,2024-02-23,2024-05-23
2,500,Juan diaz,532041134,jj-diaz,https://media.talentpitch.co/resumes/3/1231283...,michael.brown123@example.net,M,2024-02-24,2024-05-24
3,600,Pedro jimenez,3379216529,pedro-jime-23,https://media.talentpitch.co/resumes/4/aws_kaj...,sophia.wilson78@example.com,NaN,2024-02-25,2024-05-25
4,700,Lisa Duarte,4969378193,lis-duar,https://media.talentpitch.co/resumes/5/aws_123...,william.davis92@example.org,F,2024-02-26,2024-05-26
5,800,Juan jose,8368708077,juanjo-munoz-2,https://media.talentpitch.co/resumes/6/temp_aw...,olivia.martinez65@example.net,M,2024-02-27,2024-05-27
6,900,Ana beltran,1230295606,anna-beltran,https://media.talentpitch.co/resumes/7/2213231...,noah.miller34@example.com,F,2024-02-28,2024-05-28
7,1000,Luisa perez,8410487433,luisita-perez-22,NaN,ava.lopez21@example.org,F,2024-02-29,2024-05-29
8,2000,Jesus barreto,8751818549,jj-bbarreto,NaN,liam.anderson56@example.net,M,2024-03-01,2024-05-30
9,3000,Clara benavidez,4076184747,clara-bena,NaN,isabella.moore43@example.com,F,2024-03-02,2024-05-31


#Solve Questions

In [99]:
#import pyspark to create a SQL instance and join the tables
from pyspark.sql import SparkSession

#Create Spark Session
spark = SparkSession.builder.appName("Talentpitch").\
    config("spark.sql.catalogImplementation", "hive").\
    enableHiveSupport().getOrCreate()

#Load the csv files to the spark session
df1 = spark.read.csv("talentpitch/challenges - challenges.csv", header=True, inferSchema=True)
df2 = spark.read.csv("talentpitch/profiles - Hoja 1.csv", header=True, inferSchema=True)
df3 = spark.read.csv("talentpitch/resumes - resumes.csv", header=True, inferSchema=True)
df4 = spark.read.csv("talentpitch/users - users.csv", header=True, inferSchema=True)

#Create the temporary tables to make the queries
df1.createOrReplaceTempView("challenges")
df2.createOrReplaceTempView("profiles")
df3.createOrReplaceTempView("resumes")
df4.createOrReplaceTempView("users")

Number of challenges that was created in the last 3 months, with the objective of every challenge and with status are open, order by the most recent date of creation

In [100]:

query1=spark.sql(""" SELECT
id
,name
,description
,status
,opencall_objective
,created_at
FROM challenges
where 1=1
and year(created_at)*100+month(created_at)>=year(current_date())*100+month(current_date())-3
and status='open'
order by created_at desc
--limit 10
""")

query1.show()

+---+--------------------+--------------------+------+------------------+----------+
| id|                name|         description|status|opencall_objective|created_at|
+---+--------------------+--------------------+------+------------------+----------+
|490|Domina el arte de...|¿Sueñas con conve...|  open|         Innovador|2024-07-09|
|480|Sé el mejor prese...|¡Domina el arte d...|  open|          Artistas|2024-07-08|
+---+--------------------+--------------------+------+------------------+----------+



List of users that have his onboarding goal objective (profile table) in "be_discovered" and that have a video uploaded in the platform for the last 2 months. Keep in mind that if the user has more than one video in the resumes table you have to choose one option using this rule, the type of video is pitch_video. the video must be a valid
url and not null.

In [101]:
#import date functions to build the query
from pyspark.sql.functions import add_months, current_date

query2=spark.sql("""
select
p.user_id
,'be_discover' as main_objective
,r.video
,p.updated_at
,max(r.created_at) as created_at
from profiles p
inner join resumes r
on p.user_id=r.user_id
where 1=1
and p.onboarding_goal like '%be_discovered%'
AND p.created_at >= add_months(current_date(), -90)
and r.type='pitch_video'
and r.video is not null
group by
p.user_id
,r.video
,p.updated_at
--order by p.created_at desc

""")
query2.show()


+-------+--------------+--------------------+----------+----------+
|user_id|main_objective|               video|updated_at|created_at|
+-------+--------------+--------------------+----------+----------+
|    900|   be_discover|https://media.tal...|2024-04-15|2024-06-12|
|   2000|   be_discover|https://media.tal...|2024-04-20|2024-06-15|
|    600|   be_discover|https://media.tal...|2024-04-08|2024-06-09|
|    700|   be_discover|https://media.tal...|2024-04-09|2024-06-10|
|    500|   be_discover|https://media.tal...|2024-04-07|2024-06-08|
|    800|   be_discover|https://media.tal...|2024-04-22|2024-06-11|
|    400|   be_discover|https://media.tal...|2024-04-06|2024-06-07|
|   3000|   be_discover|https://media.tal...|2024-04-19|2024-06-17|
+-------+--------------+--------------------+----------+----------+



List of top users with highest profile views of their profiles

In [102]:
query3=spark.sql("""
select
p.user_id
,u.slug
,p.views
from profiles p
inner join users u
on p.user_id=u.id
where 1=1
order by p.views desc
limit 10

""")
query3.show()

+-------+----------------+-----+
|user_id|            slug|views|
+-------+----------------+-----+
|    900|    anna-beltran|  200|
|    800|  juanjo-munoz-2|  190|
|   3000|      clara-bena|   70|
|   1000|luisita-perez-22|   59|
|    600|   pedro-jime-23|   40|
|    500|         jj-diaz|   30|
|    700|        lis-duar|   30|
|    400| juanjo-suarez-2|   20|
|   2000|     jj-bbarreto|    1|
+-------+----------------+-----+

